# Online pipeline for Feldman lab

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import time

from spikeextractors import SpikeGLXRecordingExtractor, NwbSortingExtractor
from spiketoolkit.sortingcomponents import detect_spikes
from spiketoolkit.curation import threshold_firing_rates
from spikewidgets import plot_rasters
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

from feldman_lab_to_nwb import RapidTestingNWBConverter

In [ ]:
# Set parameters for parallelization
n_jobs = 8 # Number of concurrent jobs
chunk_mb = 2000  # Maximum amount of RAM in Mb

## 1) Load short AP and LF recordings

In [ ]:
base_path = Path("E:/Feldman/Neuropixels_Feldman/210209/SpikeGLX")
session_name = "LR_210209_g0"

ap_bin_path = base_path / session_name / f"{session_name}_imec0" / f"{session_name}_t0.imec0.ap.bin"
lf_bin_path = base_path / session_name / f"{session_name}_imec0" / f"{session_name}_t0.imec0.lf.bin"
nidq_file_path = base_path / session_name / f"{session_name}_t0.nidq.bin"

nwbfile_path = f"E:/Feldman/rapid_testing_{session_name}_new.nwb"

In [ ]:
recording_ap = SpikeGLXRecordingExtractor(ap_bin_path)

In [ ]:
duration = recording_ap.get_num_frames() / recording_ap.get_sampling_frequency()
fs = recording_ap.get_sampling_frequency()
print(f"Duration: {np.round(duration, 1)} s")

# 2) Quick spike detection by channel

In [ ]:
detect_spikes?

In [ ]:
t_start = time.time()
sorting_ch = detect_spikes(recording=recording_ap, n_jobs=n_jobs, chunk_mb=chunk_mb, verbose=True)
t_stop = time.time()
print(f"Elapsed time for detection: {t_stop - t_start}")

In [ ]:
print(f"Detected spikes on {len(sorting_ch.get_unit_ids())} channels")

In [ ]:
wr = plot_rasters(sorting_ch)

### (optional) Remove channels below a certain firing rate

In [ ]:
firing_rate_threshold = 0.1  # Adjusts sensitivity.

sorting_high_fr = threshold_firing_rates(
    sorting_ch,
    duration_in_frames=recording_ap.get_num_frames(),
    threshold=firing_rate_threshold, 
    threshold_sign="less"
)

In [ ]:
print(f"Detected spikes on {len(sorting_high_fr.get_unit_ids())} channels with fr > {firing_rate_threshold}")

# 3) Save spike and behavior info to NWB

In [ ]:
# Choose a sorting extractor by uncommenting one of these lines (either the basic detection or rate thresholded).
chosen_sorting = sorting_ch
#chosen_sorting = sorting_high_fr

# Run conversion to NWB.
source_data = dict(RapidTesting=dict(file_path=str(nidq_file_path)))
converter = RapidTestingNWBConverter(source_data=source_data)
metadata = converter.get_metadata()
metadata["NWBFile"].update(session_description="Rapid testing file for electrode placement.")
converter.run_conversion(
    nwbfile_path=nwbfile_path,
    metadata=metadata,
    overwrite=True  # This always creates a new file.
)

pprint("Appending spike detection...")
NwbSortingExtractor.write_sorting(
    sorting=chosen_sorting,
    save_path=nwbfile_path,
    overwrite=False  # This appends the file. True would write a new file.
)
pprint("Spike detection appended!")

# 5) View output vs. behavior in NWBWidgets 

In [ ]:
io = NWBHDF5IO(nwbfile_path, mode="r")
nwb = io.read()

nwb2widget(nwb)

In [ ]:
io.close()